In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df=pd.read_csv("/content/diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [4]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [5]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [6]:
x.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [7]:
y.head()

0    1
1    0
2    1
3    0
4    1
Name: Outcome, dtype: int64

In [8]:
# Preprocessing
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [9]:
# split the data into train and test
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=101)

Building MLP Model

In [10]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout

In [11]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [12]:
model.fit(x_train,y_train,validation_data=[x_test,y_test],batch_size=32,epochs=10)

Epoch 1/10
20/20 [==============================] - 5s 32ms/step - loss: 0.7073 - accuracy: 0.5489 - val_loss: 0.6858 - val_accuracy: 0.6039
Epoch 2/10
20/20 [==============================] - 0s 17ms/step - loss: 0.6466 - accuracy: 0.6531 - val_loss: 0.6341 - val_accuracy: 0.6429
Epoch 3/10
20/20 [==============================] - 0s 19ms/step - loss: 0.6029 - accuracy: 0.6824 - val_loss: 0.5963 - val_accuracy: 0.6818
Epoch 4/10
20/20 [==============================] - 0s 13ms/step - loss: 0.5710 - accuracy: 0.7020 - val_loss: 0.5681 - val_accuracy: 0.7143
Epoch 5/10
20/20 [==============================] - 0s 15ms/step - loss: 0.5470 - accuracy: 0.7166 - val_loss: 0.5467 - val_accuracy: 0.7468
Epoch 6/10
20/20 [==============================] - 0s 14ms/step - loss: 0.5289 - accuracy: 0.7362 - val_loss: 0.5285 - val_accuracy: 0.7597
Epoch 7/10
20/20 [==============================] - 0s 17ms/step - loss: 0.5148 - accuracy: 0.7476 - val_loss: 0.5161 - val_accuracy: 0.7468
Epoch 8/10
20

Hyperparameters Tunning:Keras Tuner

1.   How to select appropriate optimizer
2.   no.of nodes in a layer
1.   how to select no of hidden layer
2.   All in all with one model





In [13]:
!pip install -U keras-tuner

In [14]:
import kerastuner as kt

<ipython-input-14-5fd8096cdee5>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [15]:
# Hyperparameter for optimization
def build_model(hp):
  model = Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))
  optimizer = hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  return model


In [16]:

# Best Optimization

tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5)

In [17]:
tuner.search(x_train,y_train,validation_data=(x_test,y_test),epochs=10)

In [18]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

# Right no of Neurons in the given layers

In [19]:
def build_model(hp):
  model = Sequential()
  units = hp.Int('units',8,256,step=8)
  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [20]:

tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5,
                        directory='mydir',project_name='my_own_dir')

In [21]:
tuner.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10)

In [22]:
tuner.get_best_hyperparameters()[0].values

{'units': 174}

In [23]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=11,validation_data=(x_test,y_test))

Epoch 12/100
20/20 [==============================] - 0s 8ms/step - loss: 0.4836 - accuracy: 0.7655 - val_loss: 0.4822 - val_accuracy: 0.7922
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4790 - accuracy: 0.7622 - val_loss: 0.4770 - val_accuracy: 0.7987
Epoch 14/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4751 - accuracy: 0.7622 - val_loss: 0.4728 - val_accuracy: 0.7922
Epoch 15/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4725 - accuracy: 0.7638 - val_loss: 0.4677 - val_accuracy: 0.7857
Epoch 16/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4692 - accuracy: 0.7704 - val_loss: 0.4655 - val_accuracy: 0.7857
Epoch 17/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4668 - accuracy: 0.7704 - val_loss: 0.4634 - val_accuracy: 0.7857
Epoch 18/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4641 - accuracy: 0.7736 - val_loss: 0.4593 - val_accuracy: 0.7922
Epoch 

# How many hidden layer Required?

In [24]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(174,activation='relu',input_dim=8))
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(Dense(174,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [25]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5,
                        directory='mydir1',project_name='no_of_layers')

In [26]:
tuner.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10)

In [27]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1}

# All hyperparameter at one Go

In [28]:

# Import necessary libraries
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [29]:
# Define the build_model function
def build_model(hp):
    model = Sequential()
    counter = 0

    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
        if counter == 0:
            model.add(Dense(
                units=hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid', 'elu', 'leaky_relu', 'softmax', 'linear']),
                input_dim=8
            ))
            model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7])))
        else:
            model.add(Dense(
                units=hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid', 'elu', 'leaky_relu', 'softmax', 'linear'])
            ))
            model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7])))
        counter += 1

    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=hp.Choice('optimizer', values=['rmsprop', 'adam', 'sgd', 'adadelta', 'adagrad', 'adam']),
        loss='binary_cross_entropy',
        metrics=['accuracy']
    )
    return model

In [30]:

# Perform hyperparameter tuning
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='Best_Model',
    project_name='Final_HyprPar'
)


In [31]:

# Retrieve the best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters()[0]
print(best_hyperparameters.values)



IndexError: ignored